### CybORG Action Space

Without the use of wrappers,  CybORG actions need to be constructed by the agent before being passed in. If you are not interested this we suggest you skip to the wrapper tutorial.

The action space is updated every step and can be found as a dictionary in the results object. Because this dictionary is quite large, we will only print the keys below.

In [18]:
import random
import inspect
from os.path import dirname
from pprint import pprint
    
from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)
env = CybORG(scenario_generator=sg)

results = env.reset(agent='Red')
pprint(results.__str__())

action_space = results.action_space


env is: sim
Agent Interfaces are: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
('Results:\n'
 "observation={'User0': {'Interface': [{'IP Address': "
 "IPv4Address('10.0.222.69'),\n"
 "                          'Interface Name': 'eth0',\n"
 "                          'Subnet': IPv4Network('10.0.222.64/28')}],\n"
 "           'Processes': [{'PID': 3383, 'Username': 'SYSTEM'}],\n"
 "           'Sessions': [{'Agent': 'Red',\n"
 "                         'ID': 0,\n"
 "                         'PID': 3383,\n"
 "                         'Timeout': 0,\n"
 "                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,\n"
 "                         'Username': 'SYSTEM'}],\n"
 "           'System info': {'Architecture': <Architecture.x64: 2>,\n"
 "                           'Hostname': 'User0'

The CybORG action space is divided into "actions" and "parameters". Actions represent the use of specific cyber tools (for example a network scanning tool like nmap), while parameters represent the inputs the tool requires to function (to scan the interfaces of a host with nmap, you need to provide the ip address of the host).

The "actions" are located under the 'action' key in the action_space dictionary.

In [19]:
actions = action_space['action']
pprint(actions)

{<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.Impact.Impact'>: True}


We can see that our actions are each custom classes that form the keys of the above dictionary. The values specify whether this action is currently valid. In Scenario 1b, this value will always be True.

The remaining keys in the scenario dictionary represent different classes of parameters. For example, if we examine the 'ip_address' key we will get a dictionary whose keys are the various ip_addresses on the network. The values are again booleans, which represents whether Red knows about this ip_address or not.

In [20]:
ips = action_space['ip_address']
pprint(ips)

{IPv4Address('10.0.54.146'): False,
 IPv4Address('10.0.54.148'): False,
 IPv4Address('10.0.54.152'): False,
 IPv4Address('10.0.54.155'): False,
 IPv4Address('10.0.54.157'): False,
 IPv4Address('10.0.143.148'): False,
 IPv4Address('10.0.143.149'): False,
 IPv4Address('10.0.143.152'): False,
 IPv4Address('10.0.143.153'): False,
 IPv4Address('10.0.143.156'): False,
 IPv4Address('10.0.222.65'): False,
 IPv4Address('10.0.222.68'): False,
 IPv4Address('10.0.222.69'): True,
 IPv4Address('10.0.222.72'): False,
 IPv4Address('10.0.222.75'): False,
 IPv4Address('10.0.222.78'): False}


To construct an action, we choose (or import) an action class, then instantiate it by passing in the necessary parameters.

In [21]:
import random
from CybORG.Simulator.Actions import DiscoverNetworkServices
unknown_ips = [ip for ip in ips if not ips[ip]]
ip = random.choice(unknown_ips)

action = DiscoverNetworkServices(session=0,agent='Red',ip_address=ip)

In [22]:
pprint(action)

DiscoverNetworkServices 10.0.222.78


We have deliberately chosen to scan an ip address that Red Agent doesn't know about. Although randomly guessing an ip address to scan is possible in the real world, we have decided it is out of scope for our current implementation and so this action will always fail. If you want to expose your agent to the action space, you should filter out all parameters with False values first.

In [23]:
results = env.step(action=action,agent='Red')

pprint(results.__str__())

pprint("="*76)
print(results.observation)

action is: {'Red': DiscoverNetworkServices 10.0.222.78}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': InvalidAction, 'Blue': Sleep, 'Green': Sleep}
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
('Results:\n'
 "observation={'success': <TrinaryEnum.UNKNOWN: 2>}\n"
 'done=False\n'
 'reward=0.0\n'
 'action=InvalidAction\n'
 "action_space={'action': {<class 'CybORG.Simulator.Actions.Action.Sleep'>: "
 'True,\n'
 '            <class '
 "'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: "
 'True,\n'
 '            <class '
 "'CybORG.

### Red Actions

We will now take a detailed look at Red Team's actions and understand what they do. Red's actions are listed below.

In [24]:
pprint([action.__name__ for action in actions if actions[action]])

['Sleep',
 'DiscoverRemoteSystems',
 'DiscoverNetworkServices',
 'ExploitRemoteService',
 'PrivilegeEscalate',
 'Impact']


The Sleep action does nothing and requires no parameters.

In [25]:
from CybORG.Simulator.Actions import *

action = Sleep()
results = env.step(action=action,agent='Red')
print(results.observation)

action is: {'Red': Sleep}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': Sleep, 'Blue': Sleep, 'Green': Sleep}
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
{'success': <TrinaryEnum.UNKNOWN: 2>}


The DiscoverRemoteSystems action represents a ping sweep and takes in a subnet parameter to return all ips active on that subnet. Note how we pull the 

In [26]:
subnets = action_space['subnet']
known_subnets = [subnet for subnet in subnets if subnets[subnet]]
subnet = known_subnets[0]

action = DiscoverRemoteSystems(subnet = subnet, session=0,agent='Red')
results = env.step(action=action,agent='Red')
pprint(results.observation)

action is: {'Red': DiscoverRemoteSystems 10.0.222.64/28}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': DiscoverRemoteSystems 10.0.222.64/28, 'Blue': Sleep, 'Green': Sleep}
--> in actions
In simcontroller execute action
action is: DiscoverRemoteSystems 10.0.222.64/28 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
{'10.0.222.68': {'Interface': [{'IP Address': IPv4Address('10.0.222.68'),
                                'Subnet': IPv4Network('

The DiscoverNetworkServices action represents a port scan and takes in an ip address parameter to return a list of open ports and their respective services. These will be represented in the observation as new connections. The Red team must have discovered the ip address using the DiscoverRemoteSystems action in order for this action to succeed.

In [27]:
known_ips = [ip for ip in ips if ips[ip]]
ip = random.choice(known_ips)
action = DiscoverNetworkServices(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

action is: {'Red': DiscoverNetworkServices 10.0.222.69}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': DiscoverNetworkServices 10.0.222.69, 'Blue': Sleep, 'Green': Sleep}
--> in actions
In simcontroller execute action
action is: DiscoverNetworkServices 10.0.222.69 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
{'10.0.222.69': {'Interface': [{'IP Address': IPv4Address('10.0.222.69')}],
                 'Processes': [{'Connections': [{'lo

The ExploitRemoteService represents the use of a service exploit to obtain a reverse shell on the host. It requires an ip address as an input parameter and creates a new shell on the target host. 

CybORG actually models several different types of real-world exploits and this action chooses between them depending on the services available and the operating system of the host. This action will only ever succeed if the host's ip address has been discovered by Red team.

Usually the shell created by this action will be a shell with user privileges, but some exploits, such as EternalBlue, give SYSTEM access to a windows machine. In this case, performing the Privilege Escalation action afterwards is unnecessary, although our rules-based agents always will.

In [28]:
action = ExploitRemoteService(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

action is: {'Red': ExploitRemoteService 10.0.222.69}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': ExploitRemoteService 10.0.222.69, 'Blue': Sleep, 'Green': Sleep}
--> in actions
In simcontroller execute action
action is: ExploitRemoteService 10.0.222.69 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
{'10.0.222.69': {'Interface': [{'IP Address': IPv4Address('10.0.222.69')}],
                 'Processes': [{'Connections': [{'Status': <Process

The PrivilegeEscalate represents the use of malware to establish a privileged shell with root (Linux) or SYSTEM (Windows) privileges. This action requires a user shell to be on the target host.

This action has the potential to reveals information about hosts on other subnets, which can then be scanned and exploited.

In [29]:
hostname = results.observation[str(ip)]['System info']['Hostname']
action = PrivilegeEscalate(hostname=hostname,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

action is: {'Red': PrivilegeEscalate User0}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6714e0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670100>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c670040>}
action is: {'Red': PrivilegeEscalate User0, 'Blue': Sleep, 'Green': Sleep}
--> in actions
In simcontroller execute action
action is: PrivilegeEscalate User0 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.222.69'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.22

The Impact action represents the degredation of services. It requires a hostname input parameter, but will only work on the 'OpServer0' host on the Operational subnet and needs to be continually run in order to have an ongoing effect.

In [30]:
from CybORG.Agents import B_lineAgent

results = env.reset(agent='Red')
obs = results.observation
action_space = results.action_space
agent = B_lineAgent()

while True:
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    if action.__class__.__name__ == 'Impact':
        print(action)
        print(obs)
        break

Agent Interfaces are: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69fdf0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69ff40>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69fbe0>}
action is: {'Red': DiscoverRemoteSystems 10.0.255.32/28}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69fdf0>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69ff40>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c69fbe0>}
action is: {'Red': DiscoverRemoteSystems 10.0.255.32/28, 'Blue': Sleep, 'Green': Sleep}
--> in actions
In simcontroller execute action
action is: DiscoverRemoteSystems 10.0.255.32/28 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>
--> in end turn actions
In simcontroller execute action
actio

## Blue Actions

We will now take a look at Blue Team's actions and how they interact with those of Red Team.

In [31]:
env = CybORG(sg, agents={'Red':B_lineAgent()})
results = env.reset('Blue')
actions = results.action_space['action']

pprint([action.__name__ for action in actions if actions[action]])

env is: sim
Agent Interfaces are: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
['Sleep', 'Monitor', 'Analyse', 'Remove', 'Misinform', 'Restore', 'Isolate']


Similar to Red Team, the sleep action for Blue Team has no effect. However, like all Blue Team actions it does have passive monitoring capabilities as explained in the observation tutorial.

In [32]:
action = Sleep()

for i in range(4):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 2:
        # The particular obs we want
        pprint(obs)

action is: {'Blue': Sleep}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
action is: {'Blue': Sleep, 'Green': Sleep, 'Red': DiscoverRemoteSystems 10.0.158.160/28}
--> in actions
In simcontroller execute action
action is: DiscoverRemoteSystems 10.0.158.160/28 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
action is: {'Blue': Sleep}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.Agen

As explained by the Observation Tutorial, the Analyse action can detect malware files on a single host. This mimics the use of a malware-detection tool such as DensityScout. Like all of Blue's actions, it requires a hostname parameter.

We can see below that the action discovers malware on 'User1' as well as the passive monitoring picking up an exploit used Enterprise 1.

In [33]:
action = Analyse(hostname='User1',session=0,agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 1:
        pprint(obs)

action is: {'Blue': Analyse User1}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
action is: {'Blue': Analyse User1, 'Green': Sleep, 'Red': DiscoverNetworkServices 10.0.250.46}
--> in actions
In simcontroller execute action
action is: Analyse User1 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Analyse.Analyse'>
--> in actions
In simcontroller execute action
action is: DiscoverNetworkServices 10.0.250.46 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in ob

The Remove action allows Blue Team to remove any of Red's user-level shells, simulating the act of killing it as a process. It will not remove a privileged shell. This is because privileged shells in Scenario1b are assumed to be persistent, meaning that if you remove them they will immediately come back.

We can see below that the Red agent attempts to PrivilegeEscalate, but this fails as its shell has been killed. The next turn it has to re-exploit the machine. Notice the use of the get_last_action method to work out what Red's last move was.

In [34]:
action = Remove(hostname='Enterprise1', session=0, agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    pprint(obs)
    print(73*'-')
    print(env.get_last_action('Red'))
    print(73*'*')

action is: {'Blue': Remove Enterprise1}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
action is: {'Blue': Remove Enterprise1, 'Green': Sleep, 'Red': PrivilegeEscalate Enterprise1}
--> in actions
In simcontroller execute action
action is: Remove Enterprise1 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Remove.Remove'>
--> in actions
In simcontroller execute action
action is: PrivilegeEscalate Enterprise1 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations


The Restore action represents reverting the system to a known baseline. This will restore a host to the state it was at the beginning of the game. This will wipe all of Red's shells away, with the notable exception of Red's starting host 'User0', which has been baselined into the system. Although Restore is more powerful than Remove, it necessarily causes some disruption on the network so has a large negative penalty associated by using it.

Below we can see that the Analyse action detects malware on 'User1', but this disappears after restore has been used.

In [35]:
for i in range(10):
    env.step() # So Red's actions don't interfere

action = Analyse(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)
    
action = Restore(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)

action = Analyse(hostname='User1', session=0, agent='Blue')
obs = results.observation
pprint(obs)

action is: {}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
action is: {'Blue': Sleep, 'Green': Sleep, 'Red': DiscoverNetworkServices 10.0.250.35}
--> in actions
In simcontroller execute action
action is: DiscoverNetworkServices 10.0.250.35 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
action is: {}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2

### Miscellany

If you create an action that doesn't make any sense within the current scenario, CybORG will accept it, but automatically convert it to an Invalid Action. These actions automatically give a reward of -0.1.

In [36]:
action = Analyse(hostname = "Uncle Ted's Macbook", session = 1.1, agent='Cyan')

results = env.step(action=action,agent='Blue')

print(results.action)
print(results.reward)

action is: {'Blue': Analyse Uncle Ted's Macbook}
In SimulationController script
In shared EnvironmentController script
Agent interface: {'Blue': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1f00>, 'Green': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1e10>, 'Red': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f2e8c6f1ab0>}
action is: {'Blue': InvalidAction, 'Green': Sleep, 'Red': Impact Op_Server0}
--> in actions
In simcontroller execute action
action is: Impact Op_Server0 its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Impact.Impact'>
--> in end turn actions
In simcontroller execute action
action is: Monitor its type is: <class 'CybORG.Simulator.Actions.AbstractActions.Monitor.Monitor'>
--> in observations
--> in observations
InvalidAction
-13.1
